# Notebook to preprocess the Insurance Recommendation Dataset
https://www.kaggle.com/mrmorj/insurance-recommendation

In [1]:
import os
import json
import utils
import numpy as np
import pandas as pd
from scipy import sparse as sp

In [2]:
data_dir = "/media/data/Datasets/insurance-recommendation"

In [3]:
# Dataset is a single table combining all attributes
# Note: We ignore the Test.csv as it contains missing interactions
df = pd.read_csv(os.path.join(data_dir, "Train.csv"), engine="python", sep=",", encoding="latin-1")
df = df.assign(ID=df.index)
df.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,P5DA,RIBP,...,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,0,1/2/2019,F,M,1987,1X1H,2A7I,T4MS,0,0,...,0,0,0,1,0,0,0,0,0,0
1,1,1/6/2019,F,M,1981,UAOD,2A7I,T4MS,0,0,...,0,0,0,1,0,0,0,0,0,0
2,2,1/6/2013,M,U,1991,748L,QZYX,90QI,0,0,...,0,0,0,0,0,0,0,0,0,1
3,3,1/8/2019,M,M,1990,1X1H,BP09,56SI,0,0,...,0,0,0,1,0,0,0,0,0,0
4,4,1/8/2019,M,M,1990,748L,NO3L,T4MS,0,0,...,0,0,0,0,0,0,1,1,0,0


In [4]:
# extract which columns represent interacted items
with open(os.path.join(data_dir, "VariableDefinitions.txt")) as fh:
    item_columns = [line.split(" -")[0] for line in fh if "- product code" in line]
    
# ... and use these columns for the interaction matrix
interaction_matrix = df[item_columns].to_numpy()
interaction_matrix.shape

(29132, 21)

In [5]:
user_data = {
    "UserID": df["ID"],
    "Gender": [s.lower() for s in df["sex"]],
    # year when dataset was published
    "Age": 2020 - df["birth_year"],
    "Occupation": df["occupation_code"]
}

df_users = pd.DataFrame(user_data)
df_users.head()

,UserID,Gender,Age,Occupation
0,0,f,33,2A7I
1,1,f,39,2A7I
2,2,m,29,QZYX
3,3,m,30,BP09
4,4,m,30,NO3L


In [6]:
# We group users in age categories to make the data easier to handle
age_categories = [18, 25, 35, 45, 55, 100]

assigned_age_cat = np.zeros(shape=(len(df_users),) , dtype=int)
for cat in age_categories:
    assigned_age_cat += (df_users["Age"] > cat).astype(int)

assigned_age_cat[df_users["Age"].isna()] = -1
df_users["Age"] = assigned_age_cat
df_users["Age"].value_counts()

3    10766
2     8249
4     7028
5     1876
1     1209
0        4
Name: Age, dtype: int64

In [7]:
min_users_per_group = 100

# drop users of minority groups
groups_to_keep = [k for k, v in df_users["Age"].value_counts().items() if v >= min_users_per_group]
valid_users_age = df_users["Age"].isin(groups_to_keep)

groups_to_keep = [k for k, v in df_users["Occupation"].value_counts().items() if v >= min_users_per_group]
valid_users_occ = df_users["Occupation"].isin(groups_to_keep)

valid_users_mask = valid_users_age & valid_users_occ

# filter users
n_prev_users = len(df_users)
df_users = df_users.loc[valid_users_mask, :]

# adjust indices
df_users.reset_index(drop=True, inplace=True)
df_users.assign(UserID=df_users.index)

interaction_matrix = interaction_matrix[valid_users_mask, :]

n_users = len(df_users)
print(f"Dropped {n_prev_users - n_users} users due to lack in their user groups, {n_users} users remain")

Dropped 2610 users due to lack in their user groups, 26522 users remain


In [8]:
# Extracted the README file
attribute_descriptions = {
    "gender": {
        "m": "male",
        "f": "female"
    },
    "age": {
        0:  "Under 18",
        1:  "18-24",
        2:  "25-34",
        3:  "35-44",
        4:  "45-54",
        5:  "55+"
    },
    # just in case scripts require for each attribute to have a description
    "occupation": {d:d for d in df_users["Occupation"].unique()}
}

In [9]:
n_users = len(df_users)
n_items = interaction_matrix.shape[1]
n_ratings = interaction_matrix.sum()
density = n_ratings / (n_items * n_users)

# Show some statistics about the dataset
print("Number of users:", n_users)
print("Number of items:", n_items)
print("\nCounts of users per gender:")
print(df_users["Gender"].value_counts())

print("\nNumber of interactions:", n_ratings)
print(f"Density: {density:.4f}")

Number of users: 26522
Number of items: 21

Counts of users per gender:
m    18294
f     8228
Name: Gender, dtype: int64

Number of interactions: 60127
Density: 0.1080


### Data preparation
For our use-case our end-result should be binary interaction matrix, where ```1``` denotes that a user
interacted with an item, and ```0``` that they did not.

In [10]:
interaction_matrix = sp.csr_matrix(interaction_matrix)
display(interaction_matrix.shape)

# store results
storage_dir = os.path.join(data_dir, "full")
os.makedirs(storage_dir, exist_ok=True)
sp.save_npz(os.path.join(storage_dir, "interactions.npz"), interaction_matrix)

# check whether all interactions were actually kept
print("Number of interactions (again):", interaction_matrix.sum())

(26522, 21)

Number of interactions (again): 60127


In [11]:
# create new user file for our usage
df_user_info = df_users.copy()

# change column names to camel-case
rn = {cn: cn[0].lower() + cn[1:] for cn in df_user_info.columns}
df_user_info.rename(rn, inplace=True, axis=1)

df_user_info.to_csv(os.path.join(storage_dir, "user_info.csv"), index=False)
df_user_info.head()

with open(os.path.join(storage_dir, "attribute_descriptions.json"), "w") as fh:
    json.dump(attribute_descriptions, fh, indent="\t")

In [12]:
# filter users & tracks with too less interaction
min_interactions_user = 2
min_interactions_item = 5

im_all, user_info_all = utils.ensure_min_interactions(interaction_matrix, df_user_info, 
                                                      min_interactions_user, min_interactions_item)

utils.print_stats(im_all)

# store results
storage_dir = os.path.join(data_dir, f"user_gte_{min_interactions_user}_item_gte_{min_interactions_item}")
utils.store_results(storage_dir, im_all, user_info_all, attribute_descriptions)

Final shape of interactions matrix is (26519, 20)
==> 26519 users and 20 items are remaining.

Number of interactions is 60120,
which leads to a density of 0.1134.
